# Quickstart

We'll walk through Bionty's table model for species, genes, proteins, and cell types.

You'll see how to

- configure a table
- lookup identifiers of an entity
- convert between identifiers
- validate data against identifiers

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import bionty as bt
import pandas as pd

## Species


In [ ]:
species = bt.Species()

In [ ]:
species.df.head()

In [ ]:
species.df.shape[0]

You can search terms with auto-complete using a lookup object:

In [ ]:
species.lookup.white_tufted_ear_marmoset;

In [ ]:
species.df.loc["human"]

In [ ]:
species.df.loc[["pig", "human", "mouse"]]

## Gene

Next let's take a look at genes, which essentially follows the same design choices as `Species`.

The main differences are:
1. The `.dataclass` of `Gene` is species specific, therefore you will only retrieve gene entries of the specified species.
2. We implement a `.standardize` function which allows to standardizing gene names inplace in a dataframe.

In [ ]:
gene = bt.Gene(species="human")

In [ ]:
gene.df

In [ ]:
lookup = gene.lookup

In [ ]:
lookup.PDCD1;

In [ ]:
gene.df.loc["PDCD1"].head()

In [ ]:
gene.df.index

Converting between fields is currently done throught he pandas API:

In [ ]:
gene.df.loc[gene.df.index.isin(["BRCA1", "BRCA2"])]

## Protein

Protein is very similar to Gene.

In [ ]:
protein = bt.Protein(species="human")

In [ ]:
protein.fields

In [ ]:
protein.std_id

In [ ]:
uniprot_ids = ["P40925", "P40926", "O43175", "Q9UM73"]

protein.search(uniprot_ids, id_type_from="UNIPROT_ID", id_type_to="CHEMBL_ID")

## Cell Type

In [ ]:
ct = bt.CellType()

In [ ]:
dc = ct.dataclass

In [ ]:
dc.CL_0000738